In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
from datetime import date


In [ ]:
pd.set_option('display.max_columns', 30)

In [ ]:
folders = ['Входящие', 'Аптеки']
for folder in folders:
  os.makedirs(folder)

In [109]:
class Loading:
  """
  Класс для объединения данных СБиС и продаж аптек
  и выгрузки данных в файл формата excel
  """
  @staticmethod
  def __read_file(file: str, date_columns: list):
    """
    Создает датафрейм из файла в формате csv

      Args:
        file: абсолюный путь к файлу
        date_columns: список столбцов файла csv с данными типа дата
        """
    try:
      print(f'Обрабатываем файл: {file}')

      # загрузка данных
      df_current = pd.read_csv(
        file,
        encoding='windows-1251',
        on_bad_lines='skip',
        sep=';',
        parse_dates=date_columns,
        dayfirst=True
      )
      return df_current
    except UnicodeDecodeError:
        print('Ошибочная кодировка')
        return None
    except Exception as err:
        print(f'{err}')
        return None


  def read_cbis(self, folder: str, date_columns: list):
    """
    Объединяет все файлы csv с данными СБиС в один датафрейм, преобразует столбцы в датафрейме

      Args:
        folder: папка файлов
        date_columns: список столбцов файла csv с данными типа дата
      Returns:
        df: датафрейм с объединенными данными всех файлов csv из указанной папки
    """
    files = glob(folder + '/*.csv')
    df = None
    for file in files:
      try:
        df_current = self.__read_file(file, date_columns)

        num_columns = ['Сумма', 'Сумма_1', 'Сумма_НДС',
                     'Увеличение_суммы', 'НДС',
                     'Уменьшение_суммы', 'НДС']

        # преобразования дат
        if not df_current.empty:
          for col in date_columns:
            if col in df_current.columns:
              df_current[col] = pd.to_datetime(df_current[col], format='%d.%m.%Y').dt.strftime('%d.%m.%Y')

        # переименование столбцов
        df_current = df_current.rename(columns={
          'Дата': 'Дата накладной',
          'ИНН/КПП.1': 'ИНН/КПП',
          'Дата.1': 'Дата',
          'Номер.1': 'Номер 1',
          'Сумма.1': 'Сумма 1',
          'Дата.2': 'Дата',
          'НДС.1': 'НДС'
        })
        df_current.columns = [name.replace(' ', '_') for name in df_current.columns]

        # проверка наличия числовых столбцов
        existing_num_cols = [col for col in num_columns if col in df_current.columns]

        if existing_num_cols:
          # Преобразование числовых столбцов
          for col in existing_num_cols:
            try:
              df_current[col] = (
                df_current[col]
                .astype(str)
                .str.replace('\s', '', regex=True)
                .str.replace(',', '.')
                .apply(pd.to_numeric, errors='coerce')
              )
            except Exception as err:
              print(f'Ошибка преобразования числовых данных в файле {file}, в столбце {col}: {err}')

        # объединение датафреймов
        if df is None:
          print('Создаем первый датафрейм.')
          df = df_current
        else:
          print(f'Объединям датафреймы длиной {len(df)} и {len(df_current)}.')
          df = pd.concat([df, df_current], ignore_index=True)
      except UnicodeDecodeError:
        print('Ошибочная кодировка')
      except Exception as err:
        print(f'{err}')
    return df

  def combine_data(self, data, folder: str, date_columns: list):
    """
    Объединяет все файлы csv с данными аптек в один датафрейм, объединяет с данными СБиС,
    импортуриет данные в файл формата excel

      Args:
        data: датафрейм с данными СБиС
        folder: папка файлов
        date_columns: список столбцов файла csv с данными типа дата
    """
    files = glob(folder + '/*.csv')
    df = None
    for file in files:
      try:
        df_current = self.__read_file(file, date_columns)
        # преобразования дат
        if not df_current.empty:
          for col in date_columns:
            if col in df_current.columns:
              df_current[col] = pd.to_datetime(df_current[col], format='%d.%m.%Y').dt.strftime('%d.%m.%Y')

        # объединение датафреймов аптек
        if df is None:
          print('Создаем первый датафрейм.')
          df = df_current
        else:
          print(f'Объединям датафреймы длиной {len(df)} и {len(df_current)}.')
          df = pd.concat([df, df_current], ignore_index=True)
      except UnicodeDecodeError:
        print('Ошибочная кодировка')
      except Exception as err:
        print(f'{err}')
    # объединение датафрейма аптек и датафрейма СБИС
    res = df.merge(data, on='Номер накладной', how='left')
    # добавление столбца со сравнением дат
    res['Сравнение дат'] = np.where(res['Дата накладной'] != res['Дата_накладной'], 'Не совпадает!', None)
    # формирование столбцов в необходимом для выгрузки порядке
    res['Номер счет-фактуры'] = res['Номер накладной']
    res = res.rename(columns=({
      'Сумма': 'Сумма счет-фактуры',
      'Дата_накладной': 'Дата счет-фактуры'
      }))
    new_columns = [
        '№ п/п',
        'Штрих-код партии',
        'Наименование товара',
        'Поставщик',
        'Дата приходного документа',
        'Номер приходного документа',
        'Дата накладной',
        'Номер накладной',
        'Номер счет-фактуры',
        'Сумма счет-фактуры',
        'Кол-во',
        'Сумма в закупочных ценах без НДС',
        'Ставка НДС поставщика',
        'Сумма НДС',
        'Сумма в закупочных ценах с НДС',
        'Дата счет-фактуры',
        'Сравнение дат'
      ]

    self.__process_file(res[new_columns])

  @staticmethod
  def __process_file(df):
    """
    Сохраняет файл для загрузки в 1С

      Args:
        df: датафрейм для загрузки в файл формата excel.
    """
    today = date.today()
    name_folder = today.strftime('%d_%m_%Y')
    if not os.path.exists('Результат'):
      os.makedirs('Результат')
    if not os.path.exists('Результат/' + name_folder):
      os.makedirs('Результат/' + name_folder)
    excel_path = os.path.join('Результат/' + name_folder, 'data.xlsx')
    df.to_excel(excel_path, index=False, engine='openpyxl')

    print(f"Файл сохранен: {excel_path}")


In [110]:
loader = Loading()
date_columns = ['Дата', 'Дата.1', 'Дата.2']
df = loader.read_cbis('Входящие', date_columns)

Обрабатываем файл: Входящие/Входящие 03.csv
Ошибка преобразования числовых данных в файле Входящие/Входящие 03.csv, в столбце НДС: 'DataFrame' object has no attribute 'str'
Ошибка преобразования числовых данных в файле Входящие/Входящие 03.csv, в столбце НДС: 'DataFrame' object has no attribute 'str'
Создаем первый датафрейм.
Обрабатываем файл: Входящие/Входящие 01.csv
Ошибка преобразования числовых данных в файле Входящие/Входящие 01.csv, в столбце НДС: 'DataFrame' object has no attribute 'str'
Ошибка преобразования числовых данных в файле Входящие/Входящие 01.csv, в столбце НДС: 'DataFrame' object has no attribute 'str'
Объединям датафреймы длиной 95737 и 100000.
Обрабатываем файл: Входящие/Входящие 02.csv
Ошибка преобразования числовых данных в файле Входящие/Входящие 02.csv, в столбце НДС: 'DataFrame' object has no attribute 'str'
Ошибка преобразования числовых данных в файле Входящие/Входящие 02.csv, в столбце НДС: 'DataFrame' object has no attribute 'str'
Объединям датафреймы дли

In [111]:
df['Номер накладной'] = np.where(df['Контрагент'] == 'Еаптека, ООО', df['Номер'] + '/15', df['Номер'])
types_doc = ['СчФктр', 'УпдДоп', 'УпдСчфДоп', 'ЭДОНакл']
col_for_merge = ['Номер накладной', 'Сумма', 'Дата_накладной']
df_for_merge = df[col_for_merge][df['Тип_документа'].isin(types_doc)]
df_for_merge = df_for_merge.drop_duplicates('Номер накладной', keep='first')

In [112]:
date_columns = ['Дата приходного документа', 'Дата накладной']
loader.combine_data(df_for_merge, 'Аптеки', date_columns)


Обрабатываем файл: Аптеки/366.csv
Создаем первый датафрейм.
Обрабатываем файл: Аптеки/А123.csv
Объединям датафреймы длиной 5039 и 4819.
Файл сохранен: Результат/25_04_2025/data.xlsx
